#  Install, Setup gdrive and imports

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression, Lasso, LassoCV,LassoLarsCV
from sklearn import preprocessing
from sklearn.metrics import auc
from sklearn.model_selection import LeaveOneOut

In [ ]:
IN_CWR_SERVER = False 


# Files


In [ ]:
if IN_CWR_SERVER :
  projectdir = "/somedrive/folder/"
else :
  projectdir = "C:\\research\\cav\\"

#input files
# look for these files under projectdir\stats\{slideset}\
FINAL_DIL_CSV = "32_final_dilated.csv.zip"
FINAL_UDL_CSV = "32_final_undilated.csv.zip"
FINAL_CLINICAL_DIL_CSV = "eliot_dataset.csv"

#output dir and files
saveplots = True


## Configs

In [ ]:
test0 = 0 #HC vs. DY1
test1 = 1 #HC + DY1 vs. DC
test2 = 2 #HC vs. DC
TESTS = ['HC+MA.vs.DY1', 'HC+MA+DY1.vs.DC', 'HC+MA.vs.DC']

HEALTHY = ['HC', 'MA'] # ['HC', 'US', 'MA']
DESEASED = ['DC']     
DESEASED_YR1 = ['DY'] 
COHORTS = HEALTHY + DESEASED + DESEASED_YR1

# 'CAV2-DC-23' = bad cd31 scan
# 'CAV-DC-18'= misidentified scan
# 'CAV-DC-8' = no mvt file 
# 'CAV-DY1-2'= no mvt file
DISCARD_FILES = ['CAV2-DC-23', 'CAV-DC-18', 'CAV-DC-8', 'CAV-DY1-2']
    

## Selected features\variables to test

In [ ]:
#############################################################################################
########################### test2 ###########################################################
#############################################################################################

#############################################################################################
# Features with high importance from HCvsDC 
# [cav dataset] test conditions: allFeatures, Max-iter=200, niter=200
#############################################################################################
cav_t2 = ["collagen_myocar",
"sbin2area_div_count",
"in_s6",
"sbin4area_div_count",
"nuc_outside_div_tis_area",
"qbin3area_div_dab_area",
"collagen_myocar_div_myocar",
"myocyte",
"tot_nuc_count_div_tis_area",
] # 88
cav_t2 = ["in_s6",
"nuc_outside_div_tis_area",
"qbin3area_div_dab_area",
"myocyte",
"collagen_myocar_div_myocar",
"collagen_myocar" # 91 
] 

#############################################################################################
# Features with high importance from HCvsDC 
# [cav2 dataset] test conditions: allFeatures, Max-iter=200, niter=200
#############################################################################################
cav2_t2 =["dil_d_q1_area",
"in_q1",
"dil_d_s6_area",
"nuc_in_dab_div_tot_nuc_count",
"sbin5area_div_count",
"sbin3area_div_count",
"myocyte_div_myocar",
"qbin2area_div_myocyte",
"nuc_in_dab_div_nuc_outside",
"collagen_myocar",
"sbin1area_div_count",
"nuc_outside_div_tis_area" #83
#"qbin3area_div_dab_area",#81
#"qbin1area_div_count",#81
#"sbin6area_div_count",
#"sbin5area_div_dab_area" # 78
] 
#############################################################################################
########################### test1 ###########################################################
#############################################################################################

#############################################################################################
# Features with high importance from HC+DY1.vs.DC
# [cav dataset] test conditions: allFeatures, Max-iter=200, niter=200
#############################################################################################
cav_t1 =["collagen_myocar",
"myocyte",
"sbin2area_div_count",
"sbin4area_div_count",
"white_myocar",
"in_q2_n_area",
"qbin3area_div_dab_area",
"black",
"sbin1area_div_dab_area",
"nuc_outside_div_hem_area",
"tot_nuc_count_div_tis_area",
"blue_div_myocyte",
"in_s6",
"blue_myocar_collagen_myocar_div_myocyte",#93
]
#############################################################################################
# Features with high importance from HC+DY1.vs.DC
# [cav2 dataset] test conditions: allFeatures, Max-iter=200, niter=200
#############################################################################################
cav2_t1 =["myocyte_div_myocar",
"collagen_myocar",
#"qbin3area_div_dab_area",
"sbin1area_div_count",
"nuc_in_dab_div_nuc_outside",
"sbin5area_div_count",
"sbin5area_div_dab_area" #81
# "collagen_myocar_div_myocar",
# "nuc_in_dab_div_tot_nuc_count",
# "qbin2area_div_myocyte",
# "collagen",
# "in_b1_n_area" #78
]

#############################################################################################
########################### test0 ###########################################################
#############################################################################################


#############################################################################################
# Features with high importance from HC vs DY1
# [cav dataset] test conditions: allFeatures, Max-iter=200, niter=200
#############################################################################################
cav_t0 =["black",
"nuc_outside_div_tis_area",
"dab_area_div_tot_nuc_count",
"sbin3area_div_count" #67
]
#############################################################################################
# Features with high importance from HC vs DY1
# [cav2 dataset] test conditions: allFeatures, Max-iter=200, niter=200
#############################################################################################
cav2_t0 =["collagen_div_myocyte",
"qbin3area_div_count",
"nuc_in_dab_div_nuc_outside",
"myocyte_div_myocar",
"sbin4area_div_dab_area",#75
#"blue_myocar",
#"dil_d_q1_area" #74
#"qbin4area_div_count",
#"qbin2area_div_count",
#"in_dab_div_dab_area" #71
]



# Functions

## General Functions for Lasso Regression

In [ ]:

def roc_curve(y_true, y_prob, thresholds):
    fpr = []
    tpr = []

    for threshold in thresholds:
        y_pred = np.where(y_prob >= threshold, 1, 0)

        fp = np.sum((y_pred == 1) & (y_true == 0))
        tp = np.sum((y_pred == 1) & (y_true == 1))

        fn = np.sum((y_pred == 0) & (y_true == 1))
        tn = np.sum((y_pred == 0) & (y_true == 0))

        fpr.append(fp / (fp + tn))
        tpr.append(tp / (tp + fn))

    return [fpr, tpr]


def nestedCVLassoRegression(x, y, nFold):
    skfold = model_selection.StratifiedKFold(n_splits=nFold, shuffle=True)
    #skfold = model_selection.LeaveOneOut() 

    mean_auc = []
    mean_fpr = []
    mean_tpr = []
    mean_imp = [] #added for importance
    for i, (train, test) in enumerate(skfold.split(x, y)):
        xtrain, ytrain = x[train, :], y[train]
        xval, yval = x[test, :], y[test]

        estimator = LassoCV(cv=nFold, max_iter=MAX_ITER, tol=0.001, n_jobs=-1).fit(xtrain, ytrain, )
        #estimator = LassoCV(cv=nFold, max_iter=1000000, tol=0.001, n_jobs=-1).fit(xtrain, ytrain, )
        #estimator = LassoLarsCV(cv=nFold, max_iter=1000000).fit(xtrain, ytrain, )
        importance = np.abs(estimator.coef_)
        mean_imp.append(importance)

        try:
            estimator = Lasso(estimator.alpha_)
            estimator.fit(xtrain, ytrain)
            y_prob_val = estimator.predict(xval)

            ths = np.linspace(np.min(y_prob_val), np.max(y_prob_val), num=11)
            fpr, tpr = roc_curve(yval, y_prob_val, ths)

            mean_fpr.append([fpr])
            mean_tpr.append([tpr])
            mean_auc.append(auc(fpr, tpr))
        except:
            ...

    mean_auc = np.mean(mean_auc)
    mean_fpr = np.mean(mean_fpr, axis=0)
    mean_tpr = np.mean(mean_tpr, axis=0)
    mean_imp = np.mean(mean_imp, axis=0)
    return mean_auc, mean_fpr, mean_tpr, mean_imp


def runNestedCVMultipleTimes(x, y, k, nIter):
    mean_aucs = []
    mean_fprs = []
    mean_tprs = []
    mean_imps = [] # added for importances
    scaler = preprocessing.StandardScaler()
    x = scaler.fit_transform(x)

    for i in range(nIter):
        mean_auc, mean_fpr, mean_tpr, mean_imp = nestedCVLassoRegression(x, y, k)
        mean_aucs.append(mean_auc)
        mean_fprs.append(mean_fpr)
        mean_tprs.append(mean_tpr)
        mean_imps.append(mean_imp)

    std_aucs = np.std(mean_aucs)
    mean_aucs = np.mean(mean_aucs)
    mean_fprs = np.mean(mean_fprs, axis=0)[0, :]
    mean_tprs = np.mean(mean_tprs, axis=0)[0, :]
    mean_imps = np.mean(mean_imps, axis=0)

    return mean_aucs, std_aucs, mean_tprs, mean_fprs, mean_imps



## Load all data and clean dataframe

In [ ]:
def load_data_clean(testid, CSV_FILE, slideset='cav'):
    STATDIR = f"{projectdir}stats\\{slideset}\\"
    ###### 
    print(f"loading {STATDIR}{CSV_FILE} to test {TESTS[testid]} ")
    df = pd.read_csv(f'{STATDIR}{CSV_FILE}')
    # move control column to last
    df.drop(columns=['control'], inplace=True, errors='ignore') # remove this column 
    #TESTS = ['HC-US-MA_vs_DY1', 'HC-US-MA-DY1_vs_DC', 'HC-US-MA_vs_DC']
    if testid == 0 : #HC+MA vs. DY1
        df['control'] = np.where(df.cohort.isin(HEALTHY), 0, np.where(df.cohort.isin(DESEASED_YR1), 1, 2))
    elif testid == 1 : #HC+MA + DY1 vs. DC
        df['control'] = np.where(df.cohort.isin(HEALTHY + DESEASED_YR1), 0, np.where(df.cohort.isin(DESEASED), 1, 2)) 
    elif testid == 2 : #HC+MA vs. DC
        df['control'] = np.where(df.cohort.isin(HEALTHY), 0, np.where(df.cohort.isin(DESEASED), 1, 2))
    else:
        print("no test")


    # clean up dataset
    df.dropna(axis='columns', how='all', inplace=True) # drop all columns that has no value
    df = df.loc[:, (df != 0).any(axis=0)] # drop all columns that contain only zeros
    df.drop(df.loc[df['fname'].isin(DISCARD_FILES)].index, inplace=True) # drop rows of files identified to discard
    df.replace(np.nan, 0, inplace=True) # replace empty cells with zero
    df.drop(columns=['cohort', 'fname'], inplace = True) # drop columns with alpha numeric values
    # Select rows controls marked 0 or 1 based on testid
    df = df.loc[(df['control'].isin([0,1]))] 
    return df



## Filter features of interest

In [ ]:
def selectFeatures(dataframe, selected_vars):
    if selected_vars != None :
        dataframe = dataframe.loc[:, selected_vars + ["control"]]
    data = dataframe.values
    print(f"dataframe shape :{data.shape}")
    x, y = data[:, :-1], data[:, -1]
    feature_names = dataframe.columns.values.tolist()
    return x, y, feature_names[:-1]


## Silently Run NestedCV

In [ ]:
def silentlyRunNestedCVMultipleTimes(xd, yd, nFold, nIter) :
    warnings.filterwarnings('ignore') # go slient
    print("Ignoring warnings!")
    mean_auc3d, std_auc3d, tprs_3d, fprs_3d, mean_imps = runNestedCVMultipleTimes(xd, yd, nFold, nIter)
    warnings.filterwarnings('default') # reset back to normal
    return mean_auc3d, std_auc3d, tprs_3d, fprs_3d, mean_imps


## Plot Graph

In [ ]:
def plot_roc(testid, idsd, mean_aucd, std_aucd, tprs_d, fprs_d, mean_imps, selected_vars, slideset):
    PLOTDIR = f"{projectdir}plot\\{slideset}\\"
    SET = 'SEL'
    if selected_vars == None :
        SET = 'ALL'
    fig1, ax = plt.subplots(figsize=(10,10))
    ax.plot(fprs_d, tprs_d, color='r', linestyle=ls, label=r'%s: (AUC = %0.2f $\pm$ %0.2f)' % ('Multivariable logistic regression', mean_aucd, std_aucd), lw=lw, alpha=alpha)

    ax.plot([0, 1], [0, 1], linestyle=ls, lw=lw, color='black', label='Chance', alpha=alpha)
    ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05], title=f"Receiver operating characteristic (ROC) curve for {TESTS[testid]}; {SET} features", xlabel='1-Specificity', ylabel='Sensitivity')
    ax.legend(loc="lower right")
    plt.show()
    fig1.tight_layout()
    if saveplots :
        fig1.savefig(f'{PLOTDIR}{slideset}-{TESTS[testid]}-{SET}-lasso-roc.png')

    # bar graph
    data_tuples = list(zip(idsd,mean_imps))
    df = pd.DataFrame(data_tuples, columns=['feature','importance'])
    df = df[df['importance'] > 0.01]
    df.sort_values('importance', inplace=True, ascending=[False])
    fig2, ax = plt.subplots(figsize=(15,10))
    df.plot.bar(title=f"Feature importances via coefficients (sorted) for {TESTS[testid]}",x='feature',y='importance',figsize=(15,5), ax=ax)
    plt.show()
    fig2.tight_layout()
    if saveplots :
        fig2.savefig(f'{PLOTDIR}{slideset}_{TESTS[testid]}-{SET}_importance.png')
    #print(df.feature.to_string(index=False))    
    df.feature.to_csv(f'{PLOTDIR}{slideset}_{TESTS[testid]}-{SET}_importance.txt', index=False)
    return

# Main

In [ ]:
nFold = 3
nIter = 200 #10 #200
alpha = 0.8
MAX_ITER = 200 #10#200
lw = 1.5
ls = '--'

statFile = FINAL_DIL_CSV

def roc(test, slideset, selected_vars=None) :
    df = load_data_clean(test, statFile, slideset)
    df.to_csv(f"{projectdir}/combined.csv.zip", index=False, compression=dict(method='zip', archive_name=f'combined.csv'))
    xd, yd, idsd = selectFeatures(df, selected_vars)
    mean_aucd, std_aucd, tprs_d, fprs_d, mean_imps = silentlyRunNestedCVMultipleTimes(xd, yd, nFold, nIter)
    plot_roc(test, idsd, mean_aucd, std_aucd, tprs_d, fprs_d, mean_imps, selected_vars, slideset)
    return

def rocAll(test, selected_vars=None) :
    dfcav = load_data_clean(test, statFile, 'cav')
    dfcav2 = load_data_clean(test, statFile, 'cav2')
    frames = [dfcav, dfcav2]
    df = pd.concat(frames)

    
    df.to_csv(f"{projectdir}/combined.csv.zip", index=False, compression=dict(method='zip', archive_name=f'combined.csv'))

    xd, yd, idsd = selectFeatures(df, selected_vars)
    mean_aucd, std_aucd, tprs_d, fprs_d, mean_imps = silentlyRunNestedCVMultipleTimes(xd, yd, nFold, nIter)
    plot_roc(test, idsd, mean_aucd, std_aucd, tprs_d, fprs_d, mean_imps, selected_vars, 'all')
    return

# roc(test2, 'cav', None)
# roc(test2, 'cav', cav_t2)
# roc(test2, 'cav2', None)
# roc(test2, 'cav2', cav2_t2)

# roc(test1, 'cav', None) 
# roc(test1, 'cav', cav_t1)
# roc(test1, 'cav2', None)
# roc(test1, 'cav2', cav2_t1)

# roc(test0, 'cav', None) 
# roc(test0, 'cav', cav_t0)
# roc(test0, 'cav2', None)
# roc(test0, 'cav2', cav2_t0)

#rocAll(test2, None)
roc(test2, 'all', cav_t2 + cav2_t2)


# Learning/Testing Area